In [2]:
# ==========================================================
# Required Libraries Explanation
# ==========================================================
# • langchain → Core framework for building LLM pipelines
# • langchain-community → Provides document loaders and vector store integrations
# • langchain-openai → Enables GPT models and OpenAI embeddings
# • chromadb → Vector database for storing and retrieving embeddings
# • gradio → Creates interactive web-based chatbot interface
# • pypdf → Used to read and extract text from PDF documents
# • openai → Official OpenAI API client for model communication
# ==========================================================
!pip install -U langchain langchain-community langchain-openai chromadb gradio pypdf openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5

In [1]:
"""
AI-Powered HR Assistant (RAG)
-----------------------------

This project implements a Retrieval-Augmented Generation (RAG) system
to answer questions about Nestlé’s HR policy document.

It follows the required project steps:

1. Import essential tools and set up OpenAI API.
2. Load Nestlé HR policy using PyPDFLoader and split it into chunks.
3. Create vector representations using OpenAI embeddings and store in Chroma DB.
4. Build a question-answering system using GPT-3.5 Turbo.
5. Create a prompt template to guide chatbot responses.
6. Use Gradio to build a user-friendly chatbot interface.
7. Deploy the chatbot locally.

Author: Nisha Pepsi Selvarajan
"""

# ==========================================================
# 1️⃣ Import Essential Tools and Set Up OpenAI API
# ==========================================================

import os
import gradio as gr

# ==========================================================
# Required Libraries Explanation
# ==========================================================
# • langchain → Core framework for building LLM pipelines
# • langchain-community → Provides document loaders and vector store integrations
# • langchain-openai → Enables GPT models and OpenAI embeddings
# • chromadb → Vector database for storing and retrieving embeddings
# • gradio → Creates interactive web-based chatbot interface
# • pypdf → Used to read and extract text from PDF documents
# • openai → Official OpenAI API client for model communication
# ==========================================================

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# ✅ LangChain v1 imports (no langchain.chains)
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 🔐 Set env vars FIRST (Vocareum)
os.environ["OPENAI_API_KEY"] = "voc-23680865411227239960056821ff32e2cb89.36794264"
os.environ["OPENAI_BASE_URL"] = "https://openai.vocareum.com/v1"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

# ==========================================================
# 2️⃣ Load HR Policy PDF and Split into Chunks
# ==========================================================

PDF_PATH = "nestle_hr_policy.pdf"

if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(
        f"❌ {PDF_PATH} not found. Place the PDF in the same directory as this script."
    )

loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
print(f"✅ Loaded {len(documents)} pages from HR policy.")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(documents)
print(f"✅ Created {len(chunks)} text chunks.")

# ==========================================================
# 3️⃣ Create Embeddings and Store in Chroma DB
# ==========================================================

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
vectorstore.persist()
print("✅ Vector store created and persisted locally.")

# ==========================================================
# 4️⃣ Build Question-Answering System using GPT-3.5 Turbo
#    (LangChain v1 Runnable pipeline)
# ==========================================================

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

# ==========================================================
# 5️⃣ Create Prompt Template
# ==========================================================

prompt_template = """
You are an HR policy assistant.

Use ONLY the provided context to answer the question.
If the answer is not found in the context, respond with:
"I don't know based on the HR policy document."

Context:
{context}

Question:
{question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# ==========================================================
# ✅ RAG Chain (Retriever → Prompt → GPT-3.5 → Output Parser)
# ==========================================================

def format_docs(docs):
    """
    Convert retrieved documents into a single context string
    and retain page references for citations.
    """
    context_text = "\n\n".join([d.page_content for d in docs])
    pages = sorted({d.metadata.get("page", "N/A") for d in docs})
    return context_text, pages

def rag_answer(question: str):
    """
    1) Retrieve relevant chunks from Chroma
    2) Format context
    3) Invoke LLM with prompt
    4) Return answer + citations
    """
    docs = retriever.invoke(question)
    context_text, pages = format_docs(docs)

    chain = PROMPT | llm | StrOutputParser()
    answer = chain.invoke({"context": context_text, "question": question})

    # Add source citations
    sources = "\n\nSources:\n" + "\n".join([f"- Page {p}" for p in pages])
    return answer + sources

# ==========================================================
# 6️⃣ Create Gradio Interface
# ==========================================================

def chatbot(query):
    return rag_answer(query)

interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="Ask HR Policy Question"),
    outputs=gr.Textbox(label="Answer"),
    title="Nestlé HR Policy Assistant",
    description="Ask questions about Nestlé’s HR policies."
)

# ==========================================================
# 7️⃣ Deploy Chatbot
# ==========================================================

if __name__ == "__main__":
    print("🚀 Launching HR Assistant...")
    interface.launch(share=True)

✅ Loaded 8 pages from HR policy.
✅ Created 20 text chunks.


/tmp/ipython-input-4004281512.py:89: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


✅ Vector store created and persisted locally.
🚀 Launching HR Assistant...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a3c733e3f435d6c822.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
